In [192]:
from sage.all import *
import pickle

sx, sy = var('sx, sy')

ox, oy = var('ox, oy')

dx = ox - sx
dy = oy - sy
r2 = (dx ** 2) + (dy ** 2)

r = sqrt(r2)
G = (1 / (2 * pi)) * log(r)

In [193]:
forget()
assume(sy-oy>0)
assume(sy > 0)
assume(sy < 1)
Ia = integral(G, sx, -1, 1)

In [194]:
integral(Ia, sy, -1, 1)

-1/4*(2*(oy - sy)*arctan(-(ox + 1)/(oy - sy)) - 2*(oy - sy)*arctan(-(ox - 1)/(oy - sy)) - ox*log(abs(ox^2 + oy^2 - 2*oy*sy + sy^2 + 2*ox + 1)) + ox*log(abs(ox^2 + oy^2 - 2*oy*sy + sy^2 - 2*ox + 1)) - log(ox^2 + oy^2 - 2*oy*sy + sy^2 + 2*ox + 1) - log(ox^2 + oy^2 - 2*oy*sy + sy^2 - 2*ox + 1) + 4)/pi

In [127]:
def compute(f):

    integrand = G * f
    print('integrand', integrand)
    
    # The opposite assumption that `sy - oy < 0` results in the same solution
    # so this is sufficient.
    forget()
    assume(sy-oy>0)
    assume(sy < 1)
    assume(sy > -1)
    Ia = integral(integrand, sx, -1, 1)
    print('sx integral', Ia)
    
    assume(ox < 1)
    assume(ox > -1)
    assume(oy > -1)
    coincident = integral(Ia, sy, -1, 1)
    print('coincident', coincident)
    
    forget()
    # Integrating in any zone entirely outside the source region seems to give the same solution.
    assume(ox > 1)
    assume(oy > 1)
    nearfield = integral(Ia, sy, -1, 1)
    print('nearfield', nearfield)
    
    return coincident, nearfield

In [11]:
C, N = compute((1-sx) * (1-sy))

integrand 1/2*(sx - 1)*(sy - 1)*log(sqrt((ox - sx)^2 + (oy - sy)^2))/pi
sx integral -1/8*(4*((ox - 1)*oy - (ox - 1)*sy)*arctan(-(ox + 1)/(oy - sy)) - 4*((ox - 1)*oy - (ox - 1)*sy)*arctan(-(ox - 1)/(oy - sy)) - (ox^2 - oy^2 + 2*oy*sy - sy^2 - 2*ox)*log(abs(ox^2 + oy^2 - 2*oy*sy + sy^2 + 2*ox + 1)) + (ox^2 - oy^2 + 2*oy*sy - sy^2 - 2*ox)*log(abs(ox^2 + oy^2 - 2*oy*sy + sy^2 - 2*ox + 1)) + 4*ox + 3*log(ox^2 + oy^2 - 2*oy*sy + sy^2 + 2*ox + 1) + log(ox^2 + oy^2 - 2*oy*sy + sy^2 - 2*ox + 1) - 8)*(sy - 1)/pi
coincident 1/96*(16*pi*ox*oy^3 - 48*pi*ox*oy^2 - 16*pi*oy^3 + 8*((ox - 1)*arctan((oy - 1)/(ox + 1)) - (ox - 1)*arctan((oy - 1)/(ox - 1)))*oy^3 + 48*pi*oy^2 - 24*((ox - 1)*arctan((oy - 1)/(ox + 1)) - (ox - 1)*arctan((oy - 1)/(ox - 1)))*oy^2 - 8*(3*(ox - 1)*arctan((ox + 1)/(oy - 1)) - 3*(ox - 1)*arctan((ox - 1)/(oy - 1)) + (ox^3 - 3*ox^2 - 9*ox - 5)*arctan((oy - 1)/(ox + 1)) - (ox^3 - 3*ox^2 + 3*ox - 1)*arctan((oy - 1)/(ox - 1)) - ox - 4)*oy + 8*(ox + 4)*oy - 8*(9*(ox - 1)*oy + 5*ox - 5)*a

In [41]:
right = limit(C, ox=1)
left = limit(C, ox=-1)
top = limit(C, oy=1)
bottom = limit(C, oy=-1)

In [55]:
edge_pts = [
    [1,-1],[1,0],[1,1],
    [0,-1],[0,1],
    [-1,-1],[-1,0],[-1,1]
]

In [112]:
check = 1.1e-8
C.subs(ox=1.0 + check, oy=1.0 + check).n()

0.405077572908000

In [98]:
np.sqrt(np.finfo(np.float64).eps)

1.4901161193847656e-08

In [80]:
left.subs(oy=-99999999999999/100000000000000).n()

0.595403741307922

In [83]:
left.limit(oy=-1, dir='-').n()

1.26207040797461

In [117]:
bottom.limit(ox=0).n()

0.184999955375790

In [125]:
C.subs(oy=-0.99999, ox=0).n()

-0.148340031535368

In [122]:
C.limit(oy=-1).subs(ox=0).n()

0.184999955375790

In [116]:
for p in edge_pts:
    interior_val = C.subs(ox=p[0] * (1 - 1e-5), oy=p[1] * (1 - 1e-5)).n()
    exterior_val = N.subs(ox=p[0] * (1 + 1e-5), oy=p[1] * (1 + 1e-5)).n()
    edge_val = 0
    if p[0] == 1:
        dir_ = '+' if p[1] < 0 else '-'
        edge_val = right.limit(oy = p[1], dir=dir_).n()
    elif p[0] == -1:
        dir_ = '+' if p[1] < 0 else '-'
        edge_val = left.limit(oy=p[1], dir=dir_).n()
    elif p[1] == 1:
        dir_ = '+' if p[0] < 0 else '-'
        edge_val = top.limit(ox=p[0], dir=dir_).n()
    elif p[1] == -1:
        dir_ = '+' if p[0] < 0 else '+'
        edge_val = bottom.limit(ox=p[0], dir=dir_).n()
    print(p, dir_)
    print(f'{interior_val:.3f}, {exterior_val:.3f}, {edge_val:.3f}, {interior_val - exterior_val:.2e}')

[1, -1] +
0.247, 0.247, 0.247, -1.20e-5
[1, 0] -
0.206, 0.206, 0.206, -8.75e-6
[1, 1] -
0.405, 0.405, 0.405, -9.53e-6
[0, -1] +
-0.148, -0.148, 0.185, -1.33e-5
[0, 1] -
0.206, 0.206, 0.372, -8.75e-6
[-1, -1] +
-0.071, -0.071, 0.595, -2.41e-5
[-1, 0] -
-0.148, -0.148, -0.148, -1.33e-5
[-1, 1] -
0.247, 0.247, 0.580, -1.20e-5


In [53]:
N.subs(ox = 1.0001, oy=1.0001).n()

0.405125203567794

In [54]:
C.subs(ox=0.9999, oy=0.9999).n()

0.405029935468244

In [50]:
limit(N, oy=1, taylor=True) - top

1/96*ox^4*log(ox^2 + 2*ox + 5)/pi - 1/96*ox^4*log(ox^2 + 2*ox + 1)/pi + 1/96*ox^4*log(ox^2 - 2*ox + 1)/pi - 1/96*ox^4*log(abs(ox^2 - 2*ox + 5))/pi - 1/24*ox^3*log(ox^2 + 2*ox + 5)/pi + 1/24*ox^3*log(ox^2 + 2*ox + 1)/pi - 1/24*ox^3*log(ox^2 - 2*ox + 1)/pi + 1/24*ox^3*log(abs(ox^2 - 2*ox + 5))/pi + 1/16*ox^2*log(ox^2 + 2*ox + 5)/pi + 3/16*ox^2*log(ox^2 + 2*ox + 1)/pi + 1/16*ox^2*log(ox^2 - 2*ox + 1)/pi - 5/16*ox^2*log(abs(ox^2 - 2*ox + 5))/pi + 1/3*ox + 7/6*ox*arctan(1/2*ox + 1/2)/pi - 7/6*ox*arctan(1/2*ox - 1/2)/pi - 1/6*ox*arctan(2/(ox + 1))/pi + 1/6*ox*arctan(2/(ox - 1))/pi - 17/24*ox*log(ox^2 + 2*ox + 5)/pi + 5/24*ox*log(ox^2 + 2*ox + 1)/pi - 1/24*ox*log(ox^2 - 2*ox + 1)/pi + 13/24*ox*log(abs(ox^2 - 2*ox + 5))/pi + 1/96*(16*ox*(7*arctan(1/2*ox - 1/2) - arctan(2/(ox - 1)) + 5) - 112*(ox - 1)*arctan(1/2*ox + 1/2) + 16*(ox - 1)*arctan(2/(ox + 1)) - (ox^4 - 4*ox^3 + 6*ox^2 - 68*ox - 127)*log(ox^2 + 2*ox + 5) + (ox^4 - 4*ox^3 - 18*ox^2 - 20*ox - 7)*log(ox^2 + 2*ox + 1) - (ox^4 - 4*ox^3 + 

In [32]:
(C.subs(ox=0.9999,oy=0) - C.subs(ox=-0.9999, oy=0)).n()

0.354181089841073

In [31]:
(right - left).expand().full_simplify().subs(oy=0).n()

0.354158289217195

In [5]:
import pickle
with open('data/test_integral.pkl', 'wb') as f:
    pickle.dump([coincident._sympy_(), nearfield._sympy_()], f)

In [140]:
oxn = 1
oyn = 1
dx = oxn - sx
dy = oyn - sy
r2 = (dx ** 2) + (dy ** 2)
r = sqrt(r2)
G = (1 / (2 * pi)) * log(r)

In [141]:
integrand = G * f
print('integrand', integrand)

# The opposite assumption that `sy - oy < 0` results in the same solution
# so this is sufficient.
forget()
assume(sy > 0)
assume(sy < 1)
Ia = integral(integrand, sx, -1, 1)
print('sx integral', Ia)
I = integral(Ia, sy, -1, 1)
print('I integral', I)

integrand 1/2*(sx - 1)*(sy - 1)*log(sqrt((sx - 1)^2 + (sy - 1)^2))/pi
sx integral -1/8*((sy^2 - 2*sy + 5)*log(sy^2 - 2*sy + 5) - (sy^2 - 2*sy + 1)*log(sy^2 - 2*sy + 1) - 4)*(sy - 1)/pi
I integral 1/2*(8*log(2) - 3)/pi


In [143]:
I1 = integral(Ia, sy, 0, 1)
print('I integral', I1)

I integral 1/32*(25*log(5) - 32*log(2) - 12)/pi


In [144]:
forget()
assume(sy < 0)
assume(sy > -1)
Ib = integral(integrand, sx, -1, 1)
print('sx integral', Ib)

sx integral -1/8*(sy^2*log(sy^2 - 2*sy + 5) - sy^2*log(sy^2 - 2*sy + 1) - 2*sy*log(sy^2 - 2*sy + 5) + 2*sy*log(sy^2 - 2*sy + 1) + 5*log(sy^2 - 2*sy + 5) - log(sy^2 - 2*sy + 1) - 4)*(sy - 1)/pi


In [145]:
I2 = integral(Ia, sy, -1, 0)
print('I integral', I2)

I integral -1/32*(25*log(5) - 160*log(2) + 36)/pi


In [169]:
N = 5
chebyshev_pts = [cos(pi * i / (N - 1)) for i in range(N)]

In [183]:
def interp_poly(N, i):
    vals = [0 for j in range(N)]
    vals[i] = 1
    R.<x>=PolynomialRing(RR)
    points=list(zip(chebyshev_pts, vals))
    f=R.lagrange_polynomial(points)
    return f

In [185]:
interp_poly(5, 0)

x^4 + x^3 - 0.500000000000000*x^2 - 0.500000000000000*x

In [177]:
points

[(1, 1), (1/2*sqrt(2), 0), (0, 0), (-1/2*sqrt(2), 0), (-1, 0)]